In [ ]:
import clinical_trial_extractor
import SQLConnect

data = clinical_trial_extractor.extract()
query = SQLConnect.UnmergedV1

In [ ]:
# Types of information to be collected
# Phase 1 upload
bioentity = set()
org = set()
people = set()
project = set()
# Phase 2 upload
people_org = set()
work = set()
# Phase 3 upload
work_org = set()
work_people = set()
keyword = set()

In [ ]:
# People name exception
job_title = set()
exception_names = ['NADIA NATHALIE HANSEL', 'BALAMURALI AMBATI', 'HYO-SUK LEE', 'RAZELLE KURZROCK', 'SUNG RAE KIM', 'MALCOLM K BRENNER']
job_title.update(['md', 'MMath', 'm', 'MMed', 'Ph.D.', 'Phd', 'ph.D.', 'MD,PhD', 'DNs', 'MD，PhD', 'PhD，MD', 'MD PhD', 'MBChB', 'DPhil'])

In [ ]:
# Get data from extractor
for record in data:
    bioentity.update([('ClinicalTrials.gov', entity) for entity in record['bio'] if entity])
    org.update([('ClinicalTrials.gov', entity, None) for entity in (record['aff'] + ([record['name_org'][1]] if record['name_org'] else [])) if entity])
    # Splitting the name from the JSON file may leave behind titles like PhD,
    # MD PhD, etc. The loop tries to remove these kinds of erroneous names
    new_people = record['name'] + ([record['name_org'][0]] if record['name_org'] else [])
    valid_name = set()
    for name in new_people:
        if (name == 'PhD' or name.isupper()) and name not in exception_names:
            job_title.add(name)
        else:
            valid_name.add(name)
    people.update([('ClinicalTrials.gov', None, None, entity, None, None, None, None) for entity in valid_name if entity])
    project.add(('ClinicalTrials.gov', record['title'], record['start_date']))
    if record['name_org']:
        people_org.add(record['name_org'] + (int(record['start_date'].split('-')[0]),))
    work.add((record['title'], record['start_date']))
    work_org.update([(
        (record['title'], record['start_date']),
        entity
        ) for entity in (record['aff'] + ([record['name_org'][1]] if record['name_org'] else [])) if entity])
    work_people.update([(
        (record['title'], record['start_date']),
        entity
    ) for entity in valid_name if entity])
    keyword.update([(
        (record['title'], record['start_date']),
        entity
    ) for entity in record['bio'] if entity])

In [ ]:
# Phase 1 upload
bioentity_query = [query.bioentity(entity) for entity in bioentity]
org_query = [query.org(entity) for entity in org]
people_query = [query.people(entity) for entity in people]
project_query = [query.project(entity) for entity in project]
phase_1_queries = bioentity_query + org_query + people_query + project_query
phase_1_types = ['INSERT' for _ in range(len(phase_1_queries))]
# SQLConnect.connect_and_query(phase_1_queries, phase_1_types, "UnmergedV1")

In [ ]:
# Get id data
bioentity_id_req = "SELECT bio_id, name FROM Bioentity WHERE origin_database='ClinicalTrials.gov';"
org_id_req = "SELECT org_id, name FROM Org WHERE origin_database='ClinicalTrials.gov';"
people_id_req = "SELECT people_id, name FROM People WHERE origin_database='ClinicalTrials.gov';"
project_id_req = "SELECT project_id, title, start_date FROM Project WHERE origin_database='ClinicalTrials.gov';"
id_req = [bioentity_id_req, org_id_req, people_id_req, project_id_req]
response = SQLConnect.connect_and_query(id_req, ['SELECT' for _ in range(4)], 'UnmergedV1')

In [ ]:
# Make dictionary mapping data to id
bioentity_id_dict = {}
org_id_dict = {}
people_id_dict = {}
project_id_dict = {}
for record in response[0]:
    bioentity_id_dict[record[1]] = record[0]
for record in response[1]:
    org_id_dict[record[1]] = record[0]
for record in response[2]:
    people_id_dict[record[1]] = record[0]
for record in response[3]:
    date = str(record[2].year) + '-'
    if record[2].month < 10:
        date += '0'
    date += str(record[2].month) + '-'
    if record[2].day < 10:
        date += '0'
    date += str(record[2].day)
    project_id_dict[(record[1], date)] = record[0]

In [ ]:
# Entries for queries in phase 2
people_org_id = []
work_id = []
people_org_exceptions = set()                                           # Failure Cases
work_exceptions = set()                                                 # Failure Cases
for record in people_org:
    if record[0] in people_id_dict and record[1] in org_id_dict:
        people_org_id.append((
            people_id_dict[record[0]],
            org_id_dict[record[1]],
            record[2]
        ))
    else:
        people_org_exceptions.add(record)
for record in work:
    if record in project_id_dict:
        work_id.append((project_id_dict[record], None))
    else:
        work_exceptions.add(record)


In [ ]:
# Make queries for phase 2
people_org_queries = [query.people_org(record) for record in people_org_id]
work_queries = [query.work(record) for record in work_id]
phase_2_queries = people_org_queries + work_queries
phase_2_types = ['INSERT' for _ in range(len(phase_2_queries))]
# SQLConnect.connect_and_query(phase_2_queries, phase_2_types, 'UnmergedV1')

In [ ]:
# Get work ids for each project
req = 'SELECT work_id, project_id FROM Work WHERE pub_id IS NULL;'
response = SQLConnect.connect_and_query([req], ['SELECT'], 'UnmergedV1')
work_id_dict = {} # Converts project id to work id
for record in response[0]:
    work_id_dict[record[1]] = record[0]
project_to_work_dict = {}
project_id_not_found = set()                                            # Failure Cases
for key, val in project_id_dict.items():
    if val in work_id_dict:
        project_to_work_dict[key] = work_id_dict[val]
    elif key not in work_exceptions:
        project_id_not_found.add(key)

In [ ]:
# Entries for queries in phase 3
work_org_id = []
work_people_id = []
keyword_id = []
phase_3_exceptions = set()                                              # Failure Cases
for record in work_org:
    if record[0] in project_to_work_dict and record[1] in org_id_dict:
        work_org_id.append((project_to_work_dict[record[0]], org_id_dict[record[1]]))
    elif record[0] not in work_exceptions:
        phase_3_exceptions.add(record)
for record in work_people:
    if record[0] in project_to_work_dict and record[1] in people_id_dict:
        work_people_id.append((project_to_work_dict[record[0]], people_id_dict[record[1]]))
    elif record[0] not in work_exceptions and record[1] not in job_title:
        phase_3_exceptions.add(record)
for record in keyword:
    if record[0] in project_to_work_dict and record[1] in bioentity_id_dict:
        keyword_id.append((project_to_work_dict[record[0]], bioentity_id_dict[record[1]]))
    elif record[0] not in work_exceptions:
        phase_3_exceptions.add(record)

In [ ]:
# Phase 3 upload
work_org_query = [query.work_org(record) for record in work_org_id]
work_people_query = [query.work_people(record) for record in work_people_id]
keyword_query = [query.keyword(record) for record in keyword_id]
phase_3_queries = work_org_query + work_people_query + keyword_query
phase_3_types = ["INSERT" for _ in range(len(phase_3_queries))]
# SQLConnect.connect_and_query(phase_3_queries, phase_3_types, 'UnmergedV1')